# <span style="color:green"> Import libraries<span style="color:green">

In [1]:
#Import all the required libraries
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn 
import random
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression
from sklearn.linear_model import Lasso, LassoCV, Ridge, RidgeCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import tensorflow as tf
from tensorflow import keras
from keras import layers
from tensorflow.keras.layers import Input, Dense, LeakyReLU, ReLU
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from sklearn.metrics import mean_absolute_error, mean_squared_error, explained_variance_score
from sklearn.feature_selection import f_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import pybiomart
from biomart import BiomartServer
from sklearn.feature_selection import RFECV
import xgboost as xgb

2024-12-20 20:37:03.363001: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-20 20:37:03.363078: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-20 20:37:03.364081: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-20 20:37:03.372784: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-20 20:37:04.551254: W tensorflow/compiler/tf2

# <span style="color:green"> ML for Feature Selection and Association <span style="color:green">

In [2]:
#Read the file 
df= pd.read_table("/home/Vaishnavi/Imputeted_data/GWAStutorialldlraw.raw", delim_whitespace= True)
print(df.head())
print("Shape of file", df.shape)

     FID  IID  PAT  MAT  SEX  PHENOTYPE  rs11253478_C  rs2436029:269101:C:T_T  \
0  10004    1    0    0    2         75             2                       2   
1  10005    1    0    0    1         69             2                       2   
2  10007    1    0    0    1        108             2                       2   
3  10008    1    0    0    1         94             1                       2   
4  10009    1    0    0    1         92             2                       2   

   rs2448378_T  rs2060138_A  ...  rs11137278_C  rs6559242_G  rs1110146_T  \
0            2            1  ...             0            2            2   
1            2            2  ...             2            0            2   
2            2            2  ...             2            1            1   
3            2            1  ...             2            1            2   
4            2            2  ...             2            0            1   

   rs6559257_A  rs7865267_C  rs1378954:141005656:G:A_A  

In [3]:
#X is Independent feature and y is dependent feature
X= df.iloc[:, 6:]
print("Shape of X", X.shape)
y= df.PHENOTYPE
print("Shape of y", y.shape)

Shape of X (1282, 150125)
Shape of y (1282,)


In [4]:
y

0        75
1        69
2       108
3        94
4        92
       ... 
1277    174
1278     89
1279     77
1280     41
1281     96
Name: PHENOTYPE, Length: 1282, dtype: int64

In [5]:
#Train test data split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.3, random_state=2000)
print("The dimension of X_train is {}".format(X_train.shape))
print("The dimension of X_test is {}".format(X_test.shape))

The dimension of X_train is (897, 150125)
The dimension of X_test is (385, 150125)


In [6]:
#Standardization of X_train and X_test data by Standard Scaler
scaler= StandardScaler()
X_train= scaler.fit_transform(X_train)
X_test= scaler.transform(X_test)

## R-squared function

In [7]:
def r2_score_manual(y_true, y_pred):
    # Convert inputs to NumPy arrays
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    # Calculate the mean of the actual values
    y_mean = np.mean(y_true)
    
    # Residual sum of squares
    ss_res = np.sum((y_true - y_pred) ** 2)
    
    # Total sum of squares
    ss_tot = np.sum((y_true - y_mean) ** 2)
    
    # R² score
    r2 = 1 - (ss_res / ss_tot)
    return r2

## <span style="color:green"> LASSO <span style="color:green">

In [8]:
# Create and fit the LASSOCV model select 5000 SNP
lasso_cv = linear_model.LassoCV(alphas= [0.00062], cv=5, random_state=2000)

# Fit the model to the data
lasso_cv.fit(X,y)

#Print the optimal alpha value
print ("Optimal alpha:", lasso_cv.alpha_)

# Indices of the SNPs with non-zero coefficients
lasso_snps_ldl = np.where(lasso_cv.coef_)[0]

# See how many SNPs have a non-zero coefficient
print(len(lasso_snps_ldl), "selected SNPs")

#Final SNPs
l1= X.iloc[:,lasso_snps_ldl]
print("print shape of selected snps by lasso",l1.shape)


/apps/compiler/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.057e+02, tolerance: 1.336e+02
  model = cd_fast.enet_coordinate_descent(
/apps/compiler/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.817e+02, tolerance: 1.342e+02
  model = cd_fast.enet_coordinate_descent(
/apps/compiler/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Du

Optimal alpha: 0.00062
5000 selected SNPs
print shape of selected snps by lasso (1282, 5000)


### Linear regression

In [9]:
Y= df.PHENOTYPE.values
print("Y.shape",Y.shape)
x_train, x_test, Y_train, Y_test= train_test_split(l1, Y, test_size=0.3, random_state=2000)
print(f"x_train={x_train.shape}, x_test={x_test.shape}, Y_train= {Y_train.shape}, Y_test= {Y_test.shape}")
scaler= StandardScaler()
x_train= scaler.fit_transform(x_train)
x_test= scaler.transform(x_test)

random.seed(2000)
model = linear_model.LinearRegression()
model.fit(x_train, Y_train)
y_pred = model.predict(x_test)
score= r2_score(Y_test, y_pred)
print("r2 is ", score)
r2_man = r2_score_manual(Y_test, y_pred)
print("r2 is ",r2_man)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

#f_statistic, p_value = f_regression(l1, Y)
#selected_p_indices = np.where(p_value < 5e-04)[0]
#selected_p_values = p_value[selected_p_indices]
#print("Selected p-values:")
#print(selected_p_values)
#print("Number of selected features:", len(selected_p_indices))
#selected_f_values = f_statistic[selected_p_indices]
#print("Selected F-values:")
#print(selected_f_values)
#top_f_indices = np.argsort(-selected_f_values)
#print("Indices of selected features with top F-values:")
#print(top_f_indices)
#top_f_features = l1.iloc[:, selected_p_indices[top_f_indices]]
#print("Top selected features:")
#print(top_f_features)
#lasso_LR_sigSNP=l1.iloc[:, selected_p_indices]
#lasso_LR_sigSNP
#lasso_LR_sig_col= list(lasso_LR_sigSNP.columns)
#lasso_LR_sig_col

Y.shape (1282,)
x_train=(897, 5000), x_test=(385, 5000), Y_train= (897,), Y_test= (385,)
r2 is  0.6835509848592075
r2 is  0.6835509848592075
mean_absolute_error 14.699557894419062
mean_squared_error  340.5430770142561
explained_variance_score 0.6843293027084176


### Random Forest

In [10]:
# Create a Random Forest Regressor
# Create a Random Forest Regressor
rf_regressor = RandomForestRegressor(n_estimators=100, max_depth=9, min_samples_leaf=4, min_samples_split=2, 
                                     n_jobs=70,random_state=2000)

rf_regressor.fit(x_train, Y_train)


# Make predictions on the test set using the model with selected features
y_pred= rf_regressor.predict(x_test)

score= r2_score(Y_test, y_pred)
print("r2 is ", score)
r2_man = r2_score_manual(Y_test, y_pred)
print("r2 is ",r2_man)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

r2 is  0.04964572853905869
r2 is  0.04964572853905869
mean_absolute_error 26.107939565289787
mean_squared_error  1022.7131461065229
explained_variance_score 0.05365766104291181


### Support Vector Machine

In [11]:
from sklearn.inspection import permutation_importance

svr = SVR(kernel = 'sigmoid' , C=100, epsilon= 0.01, gamma= 'scale', cache_size=800)
svr.fit(x_train, Y_train)
y_pred= svr.predict(x_test)
score= r2_score(Y_test, y_pred)
print("r2 is ", score)
r2_man = r2_score_manual(Y_test, y_pred)
print("r2 is ",r2_man)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))


r2 is  0.7034015854667354
r2 is  0.7034015854667354
mean_absolute_error 14.318335398926672
mean_squared_error  319.1810746441081
explained_variance_score 0.7042333593187404


### XGBOOST

In [12]:
regressor=xgb.XGBRegressor(learning_rate= 0.01, n_estimators=900,  max_depth=10, eval_metric='rmsle', random_state=2000)
regressor.fit(x_train, Y_train)
y_pred= regressor.predict(x_test)
score= r2_score(Y_test, y_pred)
print("r2 score",score)
r2_man = r2_score_manual(Y_test, y_pred)
print("r2 is ",r2_man)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

r2 score -0.03410800025870375
r2 is  -0.03410800025870375
mean_absolute_error 26.652027932699625
mean_squared_error  1112.8437869098063
explained_variance_score -0.0317069362126432


## <span style="color:green"> Ridge<span style="color:green">

In [13]:
# Create and fit the RidgeCV model select 5000 SNP
ridge_cv = linear_model.RidgeCV(alphas=[0.005], cv=5, scoring='neg_mean_squared_error')
ridge_cv.fit(X, y)

# Optimal alpha value
print("Optimal alpha:", ridge_cv.alpha_)

# Indices of the SNPs with non-zero coefficients
ridge_snps_ldl = np.where(ridge_cv.coef_)[0]

# Select coefficients with non-zero values
selected_features = ridge_cv.coef_ != 0

# Select coefficients of selected SNPs
coefficients = ridge_cv.coef_[selected_features]

# Get the indices of the top 5000 SNPs based on descending coefficients
top_n = 5000
sorted_indices = np.argsort(-coefficients)
selected_indices = sorted_indices[:top_n]

# Print the indices of selected SNPs and the number of selected SNPs
print("Selected SNPs indices based on top coefficients:", selected_indices)
print(len(selected_indices), "selected SNPs")

#Maximum Coefficient
print("Maximum value of coefficient", coefficients.max())

#Final SNPs
R1= X.iloc[:,selected_indices]
print("print shape of selected snps by Ridge",R1.shape)


Optimal alpha: 0.005
Selected SNPs indices based on top coefficients: [108972  11284  94133 ...  17089 134652  72090]
5000 selected SNPs
Maximum value of coefficient 0.09102136750922467
print shape of selected snps by Ridge (1282, 5000)


### Linear Regression

In [14]:
Y= df.PHENOTYPE.values
print("Y.shape",Y.shape)
x_train, x_test, Y_train, Y_test= train_test_split(R1, Y, test_size=0.3, random_state=2000)
print(f"x_train={x_train.shape}, x_test={x_test.shape}, Y_train= {Y_train.shape}, Y_test= {Y_test.shape}")
scaler= StandardScaler()
x_train= scaler.fit_transform(x_train)
x_test= scaler.transform(x_test)

Y.shape (1282,)
x_train=(897, 5000), x_test=(385, 5000), Y_train= (897,), Y_test= (385,)


In [15]:
random.seed(2000)
model = linear_model.LinearRegression()
model.fit(x_train, Y_train)
y_pred = model.predict(x_test)
score= r2_score(Y_test, y_pred)
print("r2 is ", score)
r2_man = r2_score_manual(Y_test, y_pred)
print("r2 is ",r2_man)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

#f_statistic, p_value = f_regression(R1, Y)
#selected_p_indices = np.where(p_value < 5e-04)[0]
#selected_p_values = p_value[selected_p_indices]
#print("Selected p-values:")
#print(selected_p_values)
#print("Number of selected features:", len(selected_p_indices))
#selected_f_values = f_statistic[selected_p_indices]
#print("Selected F-values:")
#print(selected_f_values)
#top_f_indices = np.argsort(-selected_f_values)
#print("Indices of selected features with top F-values:")
#print(top_f_indices)
#top_f_features = R1.iloc[:, selected_p_indices[top_f_indices]]
#print("Top selected features:")
#print(top_f_features)
#Ridge_LR_sigSNP=R1.iloc[:, selected_p_indices]
#Ridge_LR_sigSNP
#Ridge_LR_sig_col= list(Ridge_LR_sigSNP.columns)
#Ridge_LR_sig_col

r2 is  0.8506717066436066
r2 is  0.8506717066436066
mean_absolute_error 10.274421623556924
mean_squared_error  160.69797683601143
explained_variance_score 0.8507279185220166


### Random Forest

In [16]:
# Create a Random Forest Regressor
# Create a Random Forest Regressor
rf_regressor = RandomForestRegressor(n_estimators=100, max_depth=9, min_samples_leaf=4, min_samples_split=2, 
                                     n_jobs=70,random_state=2000)

rf_regressor.fit(x_train, Y_train)


# Make predictions on the test set using the model with selected features
y_pred= rf_regressor.predict(x_test)

score= r2_score(Y_test, y_pred)
print("r2 is ", score)
r2_man = r2_score_manual(Y_test, y_pred)
print("r2 is ",r2_man)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

r2 is  0.06849734783798322
r2 is  0.06849734783798322
mean_absolute_error 25.765898278266828
mean_squared_error  1002.4261863259692
explained_variance_score 0.075142883876264


### Support Vector Machine

In [17]:
from sklearn.inspection import permutation_importance

svr = SVR(kernel = 'sigmoid' , C=100, epsilon= 0.01, gamma= 'scale', cache_size=800)
svr.fit(x_train, Y_train)
y_pred= svr.predict(x_test)
score= r2_score(Y_test, y_pred)
print("r2 is ", score)
r2_man = r2_score_manual(Y_test, y_pred)
print("r2 is ",r2_man)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))


#perm_importance= permutation_importance(svr, x_test, Y_test, n_repeats=10, random_state=2000, n_jobs=70)

#imp_score=perm_importance.importances_mean
#feature_indices = np.argsort(np.abs(imp_score))[::-1]
#print(f"Selected feature indices are {feature_indices}")

# # Specify the number of features you want to select based on magnitude
#num_selected_features = 100
#top_100=feature_indices[:num_selected_features]
#print(f"Top 100 fearure indices are {feature_indices}")
#imp_score_100=imp_score[top_100]
#print(f"Top 100 fearures permutation scores are {imp_score_100}")

#Ridge_SVR_sigSNP= R1.iloc[:,top_100]
#Ridge_SVR_sig_col= list(Ridge_SVR_sigSNP.columns)
#Ridge_SVR_sig_col

r2 is  0.8407897808398864
r2 is  0.8407897808398864
mean_absolute_error 10.647254491629846
mean_squared_error  171.3323010367937
explained_variance_score 0.8408442684719689


### XGBoost

In [18]:
regressor=xgb.XGBRegressor(learning_rate= 0.01, n_estimators=900,  max_depth=10, eval_metric='rmsle', random_state=2000)
regressor.fit(x_train, Y_train)
y_pred= regressor.predict(x_test)
score= r2_score(Y_test, y_pred)
print("r2 score",score)
r2_man = r2_score_manual(Y_test, y_pred)
print("r2 is ",r2_man)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

r2 score -0.04370920401631073
r2 is  -0.04370920401631073
mean_absolute_error 27.09141607903815
mean_squared_error  1123.1760152126865
explained_variance_score -0.03338318094447423


## <span style="color:green"> Elastic net<span style="color:green">

In [19]:
# Create and fit the Elasticnet model
elastic_cv = linear_model.ElasticNetCV(l1_ratio=[0.5],alphas=[0.045], cv=5, random_state= 2000)
elastic_cv.fit(X, y)

# Print the optimal alpha value
print ("Optimal alpha:", elastic_cv.alpha_)
print("Optimal l1_ratio: ", elastic_cv.l1_ratio_)

# Indices of the SNPs with non-zero coefficients
elastic_snps_ldl = np.where(elastic_cv.coef_)[0]

# See how many SNPs have a non-zero coefficient
print(len(elastic_snps_ldl), "selected SNPs")

#Final SNPs
E1= X.iloc[:,elastic_snps_ldl]
print("print shape of selected snps by Elastic net",E1.shape)

/apps/compiler/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.742e+02, tolerance: 1.336e+02
  model = cd_fast.enet_coordinate_descent(
/apps/compiler/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.666e+02, tolerance: 1.342e+02
  model = cd_fast.enet_coordinate_descent(
/apps/compiler/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Du

Optimal alpha: 0.045
Optimal l1_ratio:  0.5
5000 selected SNPs
print shape of selected snps by Elastic net (1282, 5000)


### Linear Regression

In [20]:
Y= df.PHENOTYPE.values
print("Y.shape",Y.shape)
x_train, x_test, Y_train, Y_test= train_test_split(E1, Y, test_size=0.3, random_state=2000)
print(f"x_train={x_train.shape}, x_test={x_test.shape}, Y_train= {Y_train.shape}, Y_test= {Y_test.shape}")
scaler= StandardScaler()
x_train= scaler.fit_transform(x_train)
x_test= scaler.transform(x_test)

Y.shape (1282,)
x_train=(897, 5000), x_test=(385, 5000), Y_train= (897,), Y_test= (385,)


In [21]:
random.seed(2000)
model = linear_model.LinearRegression()
model.fit(x_train, Y_train)
y_pred = model.predict(x_test)
score= r2_score(Y_test, y_pred)
print("r2 is ", score)
r2_man = r2_score_manual(Y_test, y_pred)
print("r2 is ",r2_man)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

#f_statistic, p_value = f_regression(E1, Y)
#selected_p_indices = np.where(p_value < 5e-04)[0]
#selected_p_values = p_value[selected_p_indices]
#print("Selected p-values:")
#print(selected_p_values)
#print("Number of selected features:", len(selected_p_indices))
#selected_f_values = f_statistic[selected_p_indices]
#print("Selected F-values:")
#print(selected_f_values)
#top_f_indices = np.argsort(-selected_f_values)
#print("Indices of selected features with top F-values:")
#print(top_f_indices)
#top_f_features = E1.iloc[:, selected_p_indices[top_f_indices]]
#print("Top selected features:")
#print(top_f_features)
#Elasticnet_LR_sigSNP=E1.iloc[:, selected_p_indices]
#Elasticnet_LR_sigSNP
#Elasticnet_LR_sig_col= list(Elasticnet_LR_sigSNP.columns)
#Elasticnet_LR_sig_col

r2 is  0.9266011671336322
r2 is  0.9266011671336322
mean_absolute_error 7.2949537183920246
mean_squared_error  78.98733507654364
explained_variance_score 0.9266095503165738


### Random Forest

In [22]:
# Create a Random Forest Regressor
# Create a Random Forest Regressor
rf_regressor = RandomForestRegressor(n_estimators=100, max_depth=9, min_samples_leaf=4, min_samples_split=2, 
                                     n_jobs=70,random_state=2000)

rf_regressor.fit(x_train, Y_train)


# Make predictions on the test set using the model with selected features
y_pred= rf_regressor.predict(x_test)

score= r2_score(Y_test, y_pred)
print("r2 is ", score)
r2_man = r2_score_manual(Y_test, y_pred)
print("r2 is ",r2_man)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

r2 is  0.10045828387770017
r2 is  0.10045828387770017
mean_absolute_error 25.43497567580952
mean_squared_error  968.0317815957836
explained_variance_score 0.10187381796567085


### Support Vector Regression

In [23]:
from sklearn.inspection import permutation_importance

svr = SVR(kernel = 'sigmoid' , C=100, epsilon= 0.01, gamma= 'scale', cache_size=800)
svr.fit(x_train, Y_train)
y_pred= svr.predict(x_test)
score= r2_score(Y_test, y_pred)
print("r2 is ", score)
r2_man = r2_score_manual(Y_test, y_pred)
print("r2 is ",r2_man)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))


r2 is  0.9489763648346148
r2 is  0.9489763648346148
mean_absolute_error 6.00284057179963
mean_squared_error  54.90851571126669
explained_variance_score 0.9489864488835495


# XGBoost

In [24]:
regressor=xgb.XGBRegressor(learning_rate= 0.01, n_estimators=900,  max_depth=10, eval_metric='rmsle', random_state=2000)
regressor.fit(x_train, Y_train)
y_pred= regressor.predict(x_test)
score= r2_score(Y_test, y_pred)
print("r2 score",score)
r2_man = r2_score_manual(Y_test, y_pred)
print("r2 is ",r2_man)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

r2 score 0.047884105014666556
r2 is  0.047884105014666556
mean_absolute_error 26.06981248731737
mean_squared_error  1024.6088976078202
explained_variance_score 0.050348135679062156


## <span style="color:green"> Mutual information <span style="color:green">

In [25]:
# Create and fit the Mutual_info model
mutual_info= mutual_info_regression(X, y, random_state=2000)
print("Length of mutal info", len(mutual_info))
mutual_info= pd.Series(mutual_info, index=df.columns[6:])

#Sort mutual_info values descending order
vdesc= mutual_info.sort_values(ascending=False)
vdesc_nonzero= vdesc[vdesc!=0]
print("Maximum value of mutualinfo", vdesc_nonzero.max)
print("Length of mutal info non zero", len(vdesc_nonzero))
k= np.where(vdesc_nonzero)
first_col= k[0]

#Selected 5000 SNPs
top5000 = 5000
mi_pos= first_col[:top5000]

#Final SNPs
M1= X.iloc[:,mi_pos]
print("print shape of selected snps by Mutual_info",M1.shape)

Length of mutal info 150125
Maximum value of mutualinfo <bound method NDFrame._add_numeric_operations.<locals>.max of rs2454962_T     8.227665e-02
rs379387_C      7.688184e-02
rs7116167_T     7.608883e-02
rs2198207_C     7.377785e-02
rs2581623_T     7.361112e-02
                    ...     
rs9315776_G     1.771523e-06
rs11855700_T    1.480320e-06
rs2251611_G     1.455906e-06
rs8042116_T     8.360537e-07
rs12128709_G    3.290196e-07
Length: 74209, dtype: float64>
Length of mutal info non zero 74209
print shape of selected snps by Mutual_info (1282, 5000)


### Linear Regression

In [26]:
Y= df.PHENOTYPE.values
print("Y.shape",Y.shape)
x_train, x_test, Y_train, Y_test= train_test_split(M1, Y, test_size=0.3, random_state=2000)
print(f"x_train={x_train.shape}, x_test={x_test.shape}, Y_train= {Y_train.shape}, Y_test= {Y_test.shape}")
scaler= StandardScaler()
x_train= scaler.fit_transform(x_train)
x_test= scaler.transform(x_test)

Y.shape (1282,)
x_train=(897, 5000), x_test=(385, 5000), Y_train= (897,), Y_test= (385,)


In [27]:
random.seed(2000)
model = linear_model.LinearRegression()
model.fit(x_train, Y_train)
y_pred = model.predict(x_test)
score= r2_score(Y_test, y_pred)
print("r2 is ", score)
r2_man = r2_score_manual(Y_test, y_pred)
print("r2 is ",r2_man)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

#f_statistic, p_value = f_regression(M1, Y)
#selected_p_indices = np.where(p_value < 5e-04)[0]
#selected_p_values = p_value[selected_p_indices]
#print("Selected p-values:")
#print(selected_p_values)
#print("Number of selected features:", len(selected_p_indices))
#selected_f_values = f_statistic[selected_p_indices]
#print("Selected F-values:")
#print(selected_f_values)
#top_f_indices = np.argsort(-selected_f_values)
#print("Indices of selected features with top F-values:")
#print(top_f_indices)
#top_f_features = M1.iloc[:, selected_p_indices[top_f_indices]]
#print("Top selected features:")
#print(top_f_features)
#MI_LR_sigSNP=M1.iloc[:, selected_p_indices]
#MI_LR_sigSNP
#MI_LR_sig_col= list(MI_LR_sigSNP.columns)
#MI_LR_sig_col

r2 is  -0.3549949068648166
r2 is  -0.3549949068648166
mean_absolute_error 30.88086267645026
mean_squared_error  1458.162651310802
explained_variance_score -0.3548642605452863


### Random Forest

In [28]:
# Create a Random Forest Regressor
# Create a Random Forest Regressor
rf_regressor = RandomForestRegressor(n_estimators=100, max_depth=9, min_samples_leaf=4, min_samples_split=2, 
                                     n_jobs=70,random_state=2000)

rf_regressor.fit(x_train, Y_train)


# Make predictions on the test set using the model with selected features
y_pred= rf_regressor.predict(x_test)

score= r2_score(Y_test, y_pred)
print("r2 is ", score)
r2_man = r2_score_manual(Y_test, y_pred)
print("r2 is ",r2_man)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

r2 is  -0.01145467568966807
r2 is  -0.01145467568966807
mean_absolute_error 26.826803813776582
mean_squared_error  1088.4656644185422
explained_variance_score -0.010302914736393998


### Support Vector Regressor

In [29]:
svr = SVR(kernel = 'sigmoid' , C=100, epsilon= 0.01, gamma= 'scale')
svr.fit(x_train, Y_train)
y_pred= svr.predict(x_test)

score= r2_score(Y_test, y_pred)
print("r2 is ", score)
r2_man = r2_score_manual(Y_test, y_pred)
print("r2 is ",r2_man)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))


r2 is  -0.45097301701422765
r2 is  -0.45097301701422765
mean_absolute_error 31.411199609285006
mean_squared_error  1561.448423717936
explained_variance_score -0.4484188686735737


### XGboost

In [30]:
regressor=xgb.XGBRegressor(learning_rate= 0.01, n_estimators=900,  max_depth=10, eval_metric='rmsle', random_state=2000)
regressor.fit(x_train, Y_train)
y_pred= regressor.predict(x_test)
score= r2_score(Y_test, y_pred)
print("r2 score",score)
r2_man = r2_score_manual(Y_test, y_pred)
print("r2 is ",r2_man)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))


r2 score -0.08999244808581452
r2 is  -0.08999244808581452
mean_absolute_error 27.327849866198253
mean_squared_error  1172.9832119347816
explained_variance_score -0.08982811053637163
